# MT5 Python Connection

Reference [documentation](https://www.mql5.com/en/docs/integration/python_metatrader5).

For safety, I will use an `.env` file to store credentials and configurations.

You can edit this file with your own data.

## Import the Required Modules and Setup of Global Variables

The code below imports the required python modules.

Bear in mind that you should have run the installation: `pip install -r requirements.txt` in the same __active__ environment used by the `ipython` kernel.

In [15]:
from datetime import datetime as dt
import os
import pprint
import pytz

import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests

from dotenv import dotenv_values
from pandas.tseries import offsets
from plotly.subplots import make_subplots

# Globals
pp = pprint.PrettyPrinter(indent=4)
config = {
    **dotenv_values(".env")
}

## Create the Connection With the Terminal

The code block below connects python to the MT5 terminal.

It is important to mention that you need to close and reopen the terminal if you restart this script. The `initialize` function does not reconnect to a previously opened terminal.

I used `PrettyPrinter` here to make the console output look tidy and nicely formatted.

In [16]:
connected = mt5.initialize(
    path=config['MT5_PATH'],               # path to the MetaTrader 5 terminal EXE file
    login=int(config['LOGIN']),            # account number as integer, not text string
    password=config['PASSWORD'],           # password
    server=config['SERVER'],               # server name as it is specified in the terminal
    timeout=int(config['TIMEOUT']),        # the time to wait for responses before failing
    portable=config['PORTABLE'] == 'True'  # portable mode, must be True or False in the .env file.
)
if not connected:
    print("initialize() failed, error code =", mt5.last_error())
    raise Exception('Could not connect')

The __PORTABLE__ parameter sets the platform to run in the Portable mode, which means that the platform will try to read configurations from the installation folder instead of the special user's home directory.

In other words, if you have a default Metatrader installation, setting this flag to __True will not work__ because there will be no configurations on the installation folder.

If you start this using the False value, the terminal will start with all your configurations in place.

In [17]:
print("Terminal Info")
pp.pprint(mt5.terminal_info()._asdict())
print("Account Info")
pp.pprint(mt5.account_info()._asdict())

Terminal Info
{   'build': 2861,
    'codepage': 0,
    'commondata_path': 'C:\\Users\\rafae\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common',
    'community_account': False,
    'community_balance': 0.0,
    'community_connection': False,
    'company': 'XP Investimentos CCTVM S/A',
    'connected': True,
    'data_path': 'C:\\Users\\rafae\\AppData\\Roaming\\MetaQuotes\\Terminal\\FB9A56D617EDDDFE29EE54EBEFFE96C1',
    'dlls_allowed': True,
    'email_enabled': False,
    'ftp_enabled': False,
    'language': 'English',
    'maxbars': 100000,
    'mqid': False,
    'name': 'MetaTrader 5 Terminal',
    'notifications_enabled': False,
    'path': 'C:\\Program Files\\MetaTrader 5 Terminal',
    'ping_last': 0,
    'retransmission': 0.0,
    'trade_allowed': True,
    'tradeapi_disabled': False}
Account Info
{   'assets': 0.0,
    'balance': 100000.0,
    'commission_blocked': 0.0,
    'company': 'MetaQuotes Software Corp.',
    'credit': 0.0,
    'currency': 'USD',
    'currency_digits': 

We are connected to the platform and are able to extract data from there.

## Understand what is happening

You are connected to the __Metatrader__ installed on your machine (called here as _terminal_) and all the commands you send to it are then, sent to the remote server that has the access to the market data.

So, we are not connecting python to the broker's server directly.

In other words the __MetaTrader5__ module allows you to operate the _terminal_ using python code.

To demonstrate that, let's add a ticker to the __Market Watch__ window programatically:

In [18]:
mt5.symbol_select('EURUSD', True) # Change this from True to False and check the terminal.

True

The code block above adds (or remove) the ticker from the __Market Watch__ window. Look for your ticker. If you added a new one, scroll to the bottom of the list.

This operation can be performed using the terminal, pressing the plus sign indicated below:

![Metatrader, market watch window](../images/metatrader/MT5_MARKET_WATCH_WINDOW.png "Metatrader, market watch window")

## Explore the API

### Getting the number of Financial Instruments available

In [19]:
n_symbols = mt5.symbols_total()
n_symbols

131

See how it matches the count in the market watch window:

![Metatrader, market watch window](../images/metatrader/MT5_MARKET_WATCH_WINDOW_SYMBOLS_COUNT.png "Metatrader, market watch window")

### Listing all Symbols Data at Once

It returns a tuple and its elements can be accessed by the tuple index like this:

In [20]:
symbols = mt5.symbols_get()
pp.pprint(symbols[0]._asdict())

{   'ask': 1.17182,
    'askhigh': 1.17739,
    'asklow': 1.17115,
    'bank': '',
    'basis': '',
    'bid': 1.1718,
    'bidhigh': 1.17736,
    'bidlow': 1.17114,
    'category': '',
    'chart_mode': 0,
    'currency_base': 'EUR',
    'currency_margin': 'EUR',
    'currency_profit': 'USD',
    'custom': False,
    'description': 'Euro vs US Dollar',
    'digits': 5,
    'exchange': '',
    'expiration_mode': 15,
    'expiration_time': 0,
    'filling_mode': 1,
    'formula': '',
    'isin': '',
    'last': 0.0,
    'lasthigh': 0.0,
    'lastlow': 0.0,
    'margin_hedged': 100000.0,
    'margin_hedged_use_leg': False,
    'margin_initial': 0.0,
    'margin_maintenance': 0.0,
    'name': 'EURUSD',
    'option_mode': 0,
    'option_right': 0,
    'option_strike': 0.0,
    'order_gtc_mode': 0,
    'order_mode': 127,
    'page': 'http://www.google.com/finance?q=EURUSD',
    'path': 'Forex\\EURUSD',
    'point': 1e-05,
    'price_change': -0.3927,
    'price_greeks_delta': 0.0,
    'pric

### Listing Symbols by Group

The `group` parameter works like a simplified "regular expression" to filter the symbols name according to its names. Examples:

\*USD\* matches all symbols containing USD in the name.

!\*USD\* matches all symbols that do __not__ contain USD in the name.

In [21]:
grouped_symbols = mt5.symbols_get(group="*USD*, *EUR*")
pp.pprint(grouped_symbols[0]._asdict())

{   'ask': 1.17182,
    'askhigh': 1.17739,
    'asklow': 1.17115,
    'bank': '',
    'basis': '',
    'bid': 1.1718,
    'bidhigh': 1.17736,
    'bidlow': 1.17114,
    'category': '',
    'chart_mode': 0,
    'currency_base': 'EUR',
    'currency_margin': 'EUR',
    'currency_profit': 'USD',
    'custom': False,
    'description': 'Euro vs US Dollar',
    'digits': 5,
    'exchange': '',
    'expiration_mode': 15,
    'expiration_time': 0,
    'filling_mode': 1,
    'formula': '',
    'isin': '',
    'last': 0.0,
    'lasthigh': 0.0,
    'lastlow': 0.0,
    'margin_hedged': 100000.0,
    'margin_hedged_use_leg': False,
    'margin_initial': 0.0,
    'margin_maintenance': 0.0,
    'name': 'EURUSD',
    'option_mode': 0,
    'option_right': 0,
    'option_strike': 0.0,
    'order_gtc_mode': 0,
    'order_mode': 127,
    'page': 'http://www.google.com/finance?q=EURUSD',
    'path': 'Forex\\EURUSD',
    'point': 1e-05,
    'price_change': -0.3927,
    'price_greeks_delta': 0.0,
    'pric

### Get Data From a Single Symbol

In [22]:
symbol_info=mt5.symbol_info("EURJPY")
pp.pprint(symbol_info._asdict())

{   'ask': 0.0,
    'askhigh': 0.0,
    'asklow': 0.0,
    'bank': '',
    'basis': '',
    'bid': 0.0,
    'bidhigh': 0.0,
    'bidlow': 0.0,
    'category': '',
    'chart_mode': 0,
    'currency_base': 'EUR',
    'currency_margin': 'EUR',
    'currency_profit': 'JPY',
    'custom': False,
    'description': 'Euro vs Yen',
    'digits': 3,
    'exchange': '',
    'expiration_mode': 7,
    'expiration_time': 0,
    'filling_mode': 1,
    'formula': '',
    'isin': '',
    'last': 0.0,
    'lasthigh': 0.0,
    'lastlow': 0.0,
    'margin_hedged': 100000.0,
    'margin_hedged_use_leg': False,
    'margin_initial': 0.0,
    'margin_maintenance': 0.0,
    'name': 'EURJPY',
    'option_mode': 0,
    'option_right': 0,
    'option_strike': 0.0,
    'order_gtc_mode': 0,
    'order_mode': 127,
    'page': 'http://www.google.com/finance?q=EURJPY',
    'path': 'Forex\\EURJPY',
    'point': 0.001,
    'price_change': 0.0,
    'price_greeks_delta': 0.0,
    'price_greeks_gamma': 0.0,
    'price_g

### Getting Ticker Rates Data

To get ticker data, you have 5 functions.

Using these two, you can specify one of the 21 timeframes (the aggregation level) available in MetaTrader.

The timeframes are enums and the names follow the rule:
```
TIMEFRAME_ + M for minutes + 1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 30
           + H for hours + 1, 2, 3, 4, 6, 8, 12
           + D for day + 1
           + W for week + 1
           + MN for month + 1
```

Example: TIMEFRAME_M30 you give you the ticker data summarized in 30 minutes interval.

__copy_rates_from__: Get bars from the MetaTrader 5 terminal starting from the specified date.

__copy_rates_from_pos__: Get bars from the MetaTrader 5 terminal starting from the specified index.

__copy_rates_range__: Get bars in the specified date range from the MetaTrader 5 terminal.

__copy_ticks_from__: Get ticks from the MetaTrader 5 terminal starting from the specified date and a number of ticks.

__copy_ticks_range__: Get ticks for the specified date range from the MetaTrader 5 terminal.

We will use the `copy_ticks_from` function in the next notebook. If you are curious, go there and check it out.

### Get Ticker Data From a Timespan
The code below retrieves `USDJPY` (US dollars vs Japanese Yens) data between 2020-01-01 and 2021-02-28 and creates a pandas `DataFrame` to manipulate the data.

In [23]:
timezone = pytz.timezone("UTC")
date_from = dt(2019,11,1, tzinfo=timezone)
date_to = dt(2021,3,31, tzinfo=timezone)
timeframe = mt5.TIMEFRAME_MN1
symbol = 'USDJPY'

rates = mt5.copy_rates_range(
    symbol,
    timeframe,
    date_from,
    date_to
)

df_rates = pd.DataFrame(rates)
df_rates['time'] = pd.to_datetime(df_rates['time'], unit='s')

df_rates

,time,open,high,low,close,tick_volume,spread,real_volume
0,2019-11-01,108.019,109.669,107.884,109.487,800424,2,0
1,2019-12-01,109.517,109.726,108.426,108.658,702030,2,0
2,2020-01-01,108.705,110.287,107.647,108.346,887299,2,0
3,2020-02-01,108.431,112.225,107.506,108.062,1333127,2,0
4,2020-03-01,107.513,111.711,101.177,107.485,4590436,0,0
5,2020-04-01,107.547,109.379,106.356,107.130,2383793,0,0
6,2020-05-01,107.118,108.085,105.985,107.785,938227,0,0
7,2020-06-01,107.674,109.848,106.071,107.923,1106720,0,0
8,2020-07-01,107.924,108.161,104.185,105.927,866830,0,0
9,2020-08-01,105.769,107.047,105.099,105.898,1009714,0,0


## Plot Data Using Plotly

The first plot is the traditional candlestick. As we get the data from the terminal, this is a good chance to see how the data matches.

As the parameters required to plot a candlestick chart match the format of the data we extract from the MT5 terminal, ploting this chart is straightforward. Almost no manipulation is required.

In [24]:
def plot_candlestick():
    fig = go.Figure(
        data=go.Candlestick(
            x=df_rates['time'],
            open=df_rates['open'],
            high=df_rates['high'],
            low=df_rates['low'],
            close=df_rates['close']
        )
    )
    fig.update_layout(
        title_text='Candlestick: USD vs YEN',
        title_font_size=24,
        plot_bgcolor='white',
    )
    fig.update_xaxes(
        showgrid=False,
        tickfont_color='gray'
    )
    fig.update_yaxes(
        gridcolor='lightgray',
        tickcolor='lightgray',
        ticks="outside",
        tickfont_color='gray'
    )
    return fig

plot_candlestick()

By default, Plotly give us a "range slider" below the x axis. You can play with it to "zoom in" a specific period.

For comparison, this is the plot from the terminal, from the same period using timeframe MN1.
![Plot from MT5 terminal](../images/charts/MT5_USDJPY_MN1.png "Plot from MT5 terminal.")

Next, let's see how to add many series to a single plot using Plotly.

In [25]:
x = df_rates['time']

series = ['open', 'close', 'high', 'low']

fig = make_subplots(
    specs=[
        [
            {"secondary_y": True} #  This tells Plotly that we will create a plot with two y axes. We will add traces to it later on.
        ]
    ]
)

for serie in series:
    fig.add_trace(
        go.Scatter(
            x=x,
            y=df_rates[serie],
            name=serie.capitalize(),
            visible=True if serie == 'low' else 'legendonly',
            mode='lines'
        ),
    )

fig.update_layout(
    title="USD vs YEN",
    title_font_size=32,
    title_font_color='rgb(255, 255, 219)',

    legend_title='Series',
    legend_title_font_size=20,
    legend_title_font_color='rgb(169, 169, 169)',
    legend_font_size=16,
    legend_font_color='rgb(169, 169, 169)',

    xaxis_showgrid=False,
    xaxis_tickfont_color='rgb(169, 169, 169)',
    
    yaxis_title_text='<b>USD vs YEN</b>',
    yaxis_title_font_color='rgb(169, 169, 169)',
    yaxis_gridcolor='rgba(137, 209, 133, 0.3)',
    yaxis_ticks='outside',
    yaxis_tickcolor='rgba(137, 209, 133, 0.5)',
    yaxis_tickfont_color='rgb(169, 169, 169)',

    paper_bgcolor='rgb(30, 30, 30)',
    plot_bgcolor='rgb(37, 37, 38)',
    width=900,
    height=600,
)

fig.show()

### Mix external data
- raw data: https://oui.doleta.gov/unemploy/csv/ar539.csv
- data map: https://oui.doleta.gov/dmstree/handbooks/402/402_4/4024c6/4024c6.pdf#ETA539
- handbook, pages 3 and 13: https://wdr.doleta.gov/directives/attach/ETAH/ETHand401_4th_s01.pdf

#### Understanding and selecting the data:
The data map brings us the metadata of the table, mapping the column numbers to acronyms:
![Data Map Report 539](../images/metadata/data_map_ar539.png "Data Map Report 539.")

The handbook gives us the content of each column explaining the acronym, as such:
![Handbook 401 ETA 539](../images/metadata/data_handbook_401_pg_13_ETA539.png "Handbook 401 ETA 539.")

For this comparison, let's use the IC data, which is found in column c3.

I wrote some logic to download the raw data file and write it to disk only once. So, the first time you run this notebook, this block of code will take a little to finish depending on your connection speed.

In [26]:
# This is for downloading and saving the raw data only once.
raw_data_file = r'./data/ar539.csv'
if not os.path.exists(raw_data_file):
    os.makedirs(r'./data', exist_ok=True)
    response = requests.get(r'https://oui.doleta.gov/unemploy/csv/ar539.csv')
    with open(raw_data_file, mode='w') as of:
        text_content = response.content.decode()
        of.write(text_content)

df_claims = pd.read_csv(
    raw_data_file,
    usecols=['st', 'c2','c3'],
    parse_dates=['c2']
).rename(
    columns={'c2':'dt_end', 'c3':'ic'}
)

df_claims_slice = df_claims[
    (df_claims['dt_end'] >= np.datetime64(date_from.replace(tzinfo=None))) &
    (df_claims['dt_end'] <= np.datetime64(date_to.replace(tzinfo=None)))].copy()

The raw data from the government's website is published weekly, but our data from MT5 is monthly. Then, the code below offsets all dates to the beginning of the month and sum all the weekly values.

In [27]:
df_claims_slice['dt_end_offset'] = df_claims_slice['dt_end'] - offsets.MonthBegin()
df_claims_slice = df_claims_slice.groupby(['dt_end_offset'], as_index=False).agg({'ic':'sum'})
df_claims_slice

,dt_end_offset,ic
0,2019-11-01,1251564
1,2019-12-01,1203863
2,2020-01-01,1288679
3,2020-02-01,822927
4,2020-03-01,15295555
5,2020-04-01,15341344
6,2020-05-01,9503121
7,2020-06-01,5681192
8,2020-07-01,5860264
9,2020-08-01,3353848


### Adding the Trace of the External Data to the Secondary Axes

In [28]:
fig.add_trace(
    go.Scatter(
        x=df_claims_slice['dt_end_offset'],
        y=df_claims_slice['ic'],
        name='Initial Claims',
        mode='lines'
    ),
    secondary_y=True, #  This tells Plotly which axis we are addressing to add the trace to.
)

fig.update_layout(
    title_text='USD vs YEN and Unemployment Benefits Claims',
    legend_x=1.08,
)

fig.update_yaxes(
    secondary_y=True, #  Same here to configure aesthetics.
    title_text="<b># Claims</b>",
    title_font_color='rgb(169, 169, 169)',

    showgrid=True,
    zeroline=False,

    gridcolor='rgba(214, 120, 101, 0.3)',
    ticks='outside',
    tickcolor='rgba(214, 120, 101, 0.5)',
    tickfont_color='rgb(169, 169, 169)',
)

fig.show()

## Conclusion

Although we can create powerful charts and financial indicators using the Metatrader platform, you can complement your analysis using this python integration.

It enables you to pull fresh data from the terminal, manipulate and join with any dataset to create richier analysis.

This is also valuable for more advanced tasks like automating trade (sending buy/sell orders) or backtesting trade strategies.